# Sprint9 アンサンブル学習、グループワーク

## この課題の目的

- アンサンブル学習について理解する

## アンサンブル学習

3種類のアンサンブル学習の効果を小さめのデータセットで確認していきます。

- ブレンディング
- バギング
- スタッキング

### 小さなデータセットの用意

以前も利用した回帰のデータセットを用意します。

[House Prices: Advanced Regression Techniques](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data)

この中の<font color='red'>train.csv</font>をダウンロードし、目的変数として<font color='red'>SalePrice</font>、説明変数として、<font color='red'>GrLivArea</font>とYearBuiltを使います。

train.csvを学習用（train）8割、検証用（val）2割に分割してください。

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error, r2_score
import itertools

In [2]:
#データの抜き出し
df = pd.read_csv('train.csv')
df = df[['GrLivArea', 'YearBuilt', 'SalePrice']]

#説明変数をX, 目的変数をyのndarrayに格納
X = df[['YearBuilt', 'GrLivArea']].values
y = df['SalePrice'].values

X = X.astype('float64')
y = y.astype('float64')

#データの分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

#対数変換
X_train_std = np.log(X_train)
y_train_std = np.log(y_train)
X_test_std = np.log(X_test)
y_test_std = np.log(y_test)

### scikit-learn

単一のモデルはスクラッチ実装ではなく、scikit-learnなどのライブラリの使用を推奨します。

[sklearn.linear_model.LinearRegression — scikit-learn 0.20.0 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)

[sklearn.svm.SVR — scikit-learn 0.20.0 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html)

[sklearn.tree.DecisionTreeRegressor — scikit-learn 0.20.0 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html)

## 【問題1】ブレンディング

ブレンディングを実装し、単一モデルより精度があがる例を<font color='red'>最低3つ</font>示してください。精度があがるとは、検証用データに対する平均二乗誤差（MSE）が小さくなることを示します。

ブレンディングとは、N個の多様なモデルを独立して学習させ、推定結果を重み付けした上で足し合わせる方法です。最も単純には平均をとります。多様なモデルとは、以下のような条件を変化させることで作り出すものです。

- 手法（例：線形回帰、SVM、決定木、ニューラルネットワークなど）
- ハイパーパラメータ（例：SVMのカーネルの種類、重みの初期値など）
- 入力データの前処理の仕方（例：標準化、対数変換、PCAなど）

重要なのはそれぞれのモデルが大きく異なることです。必ずしも単一モデルの精度が高い必要はありません。

回帰問題でのブレンディングは非常に単純であるため、scikit-learnには用意されていません。

補足

分類問題の場合は、多数決を行います。回帰問題に比べると複雑なため、scikit-learnにはVotingClassifierが用意されています。

[sklearn.ensemble.VotingClassifier — scikit-learn 0.20.0 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingClassifier.html)

考察

どういった組み合わせが良いか、どのようにすると多様なモデルが作れるかを考えてみましょう。



In [3]:
def pipe_score(X_train, X_test, y_train, y_test, model):
    '''
    モデルの学習、推定し、精度を求める関数
    
    Parameters
    --------------
    X_train: 次の形のndarray, shape (n_samples, n_features)
        学習用データの特徴量
    X_test: 次の形のndarray, shape (n_samples, n_features)
        検証用データの特徴量
    y_train: 次の形のndarray, shape (n_samples,)
        学習用データの正解データ
    y_test: 次の形のndarray, shape (n_samples, )
        検証用データの正解データ
    model ; クラス
        学習に使用するモデル

    Returns
    -------
    y_pred : 次の形のndarray, shape (n_samples, 1)
        予測したデータ
    mse : float
        MSEスコア
    r2 : float
        R2スコア
    '''
    #学習モデルのインスタンス化
    cls = model
    
    #学習
    cls.fit(X_train, y_train)
    
    #推定
    y_pred = cls.predict(X_test)
    
    #MSE
    mse = mean_squared_error(y_test, y_pred)
    
    #R2
    r2 = r2_score(y_test, y_pred)
    
    return y_pred, mse, r2

In [4]:
#線形回帰
y_linear_pred, linear_mse, linear_r2 = pipe_score(X_train_std, X_test_std, 
                                                                              y_train_std, y_test_std, LinearRegression())

#SVM
y_svm_pred, svm_mse, svm_r2 = pipe_score(X_train_std, X_test_std, 
                                                                             np.ravel(y_train_std), np.ravel(y_test_std), SVR(gamma='auto'))

#決定木
y_tree_pred, tree_mse, tree_r2 = pipe_score(X_train_std, X_test_std, 
                                                                            y_train_std, y_test_std, DecisionTreeRegressor())

In [5]:
#単一モデルの精度
score = pd.DataFrame([[linear_mse, linear_r2], 
                                         [svm_mse, svm_r2], 
                                         [tree_mse, tree_r2]],
                                        columns = ['MSE', 'R2'], 
                                         index = ['線形回帰', 'SVM', '決定木'])

score

,MSE,R2
線形回帰,0.044050,0.712701
SVM,0.053603,0.650392
決定木,0.066854,0.563969


### ・ブレンディング1

単一モデルを参考に各モデルを重み付けし、足し合わせる

In [6]:
#ブレンディング1
y_brend1_pred = (y_linear_pred.reshape(-1, 1) * 0.5 + y_svm_pred.reshape(-1, 1) * 0.3+ y_tree_pred.reshape(-1, 1) * 0.2)

#MSE
brend1_mse = mean_squared_error(y_test_std, y_brend1_pred)

#R2
brend1_r2 = r2_score(y_test_std, y_brend1_pred)

print('MSE :', brend1_mse, '\t', 'R2 :', brend1_r2)

MSE : 0.04329095860297254 	 R2 : 0.7176508501502658


### ・ブレンディング2

決定木はグリッドサーチの結果からパラメータチューニングを行い、各モデルを重み付けし、足し合わせる

In [7]:
#決定木でグリッドサーチ
tree_param_grid = {'max_depth': list(range(4, 10))}

gs = GridSearchCV(estimator=DecisionTreeRegressor(),
                 param_grid = tree_param_grid,   
                 scoring='neg_mean_squared_error', 
                  cv=5)

gs.fit(X_train_std, y_train_std)

#結果の出力
print('Test set score: {}'.format(gs.score(X_test_std, y_test_std)))
print('Best parameters: {}'.format(gs.best_params_))
print('Best cross-validation: {}'.format(gs.best_score_))

Test set score: -0.04225317393283391
Best parameters: {'max_depth': 6}
Best cross-validation: -0.04903131849032219


In [8]:
#グリッドサーチを行った結果で学習、予測
gs_y_tree_pred, _, _ = pipe_score(X_train_std, X_test_std, y_train_std, y_test_std, 
                                                                  DecisionTreeRegressor(max_depth=6))

In [9]:
#ブレンディング2
y_brend2_pred = (y_linear_pred.reshape(-1, 1) * 0.4 
                                + y_svm_pred.reshape(-1, 1) * 0.2+ gs_y_tree_pred.reshape(-1, 1) * 0.4)

#MSE
brend2_mse = mean_squared_error(y_test_std, y_brend2_pred)

#R2
brend2_r2 = r2_score(y_test_std, y_brend2_pred)

print('MSE :', brend2_mse, '\t', 'R2 :', brend2_r2)

MSE : 0.04114489381818696 	 R2 : 0.7316477582128433


### ・ブレンディング3

平均をとる

In [10]:
#ブレンディング3
y_brend3_pred = (y_linear_pred.reshape(-1, 1) 
                                 + y_svm_pred.reshape(-1, 1)  + y_tree_pred.reshape(-1, 1)) / 3

#MSE
brend3_mse = mean_squared_error(y_test_std, y_brend3_pred)

#R2
brend3_r2 = r2_score(y_test_std, y_brend3_pred)

print('MSE :', brend3_mse, '\t', 'R2 :', brend3_r2)

MSE : 0.04435330246988872 	 R2 : 0.7107221080444888


### ・ブレンディング4

SVMのkernelをpoly, rbf, sigmoidとして、それらの平均をとる。

In [11]:
#SVM(karnel='poly')
y_svm_poly_pred, _, _ = pipe_score(X_train_std, X_test_std, 
                                                                             np.ravel(y_train_std), np.ravel(y_test_std), SVR(kernel='poly', gamma='auto'))

In [12]:
#SVM(karnel='rbf')
y_svm_rbf_pred, _, _ = pipe_score(X_train_std, X_test_std, 
                                                                             np.ravel(y_train_std), np.ravel(y_test_std), SVR(kernel='poly', gamma='auto'))

In [13]:
#SVM(karnel='sigmoid')
y_svm_sigmoid_pred, _, _ = pipe_score(X_train_std, X_test_std, 
                                                                             np.ravel(y_train_std), np.ravel(y_test_std), SVR(kernel='poly', gamma='auto'))

In [14]:
#ブレンディング4
y_brend4_pred = (y_svm_poly_pred.reshape(-1, 1) 
                                 + y_svm_rbf_pred.reshape(-1, 1)  + y_svm_sigmoid_pred.reshape(-1, 1)) / 3

#MSE
brend4_mse = mean_squared_error(y_test_std, y_brend4_pred)

#R2
brend4_r2 = r2_score(y_test_std, y_brend4_pred)

print('MSE :', brend4_mse, '\t', 'R2 :', brend4_r2)

MSE : 0.04160854934107324 	 R2 : 0.7286237377953078


In [15]:
#ブレンディングの精度
brend_score = pd.DataFrame([[brend1_mse, brend1_r2], 
                                                     [brend2_mse, brend2_r2], 
                                                     [brend3_mse, brend3_r2],
                                                     [brend4_mse, brend4_r2]],
                                                    columns = ['MSE', 'R2'], 
                                                     index = ['ブレンディング1', 'ブレンディング2', 'ブレンディング3', 'ブレンディング4'])

brend_score

,MSE,R2
ブレンディング1,0.043291,0.717651
ブレンディング2,0.041145,0.731648
ブレンディング3,0.044353,0.710722
ブレンディング4,0.041609,0.728624


単一モデルでは線形回帰がMSE0.044と一番低く、R2が0.713と一番高かった。ブレンディング1~4では以下のことを行った。  

- ブレンディング1 : 単一モデルを参考に各モデルを重み付けし、足し合わせる
- ブレンディング2 : 決定木はグリッドサーチの結果からパラメータチューニングを行い、各モデルを重み付けし、足し合わせる
- ブレンディング3 : 平均をとる
- ブレンディング4 : SVMのkernelをpoly, rbf, sigmoidとする
    
結果として、平均をとったブレンディング3以外のブレンディングで、単一モデルより精度が向上した。

## 【問題2】バギング

バギングを実装し、単一モデルより精度があがる例を<font color='red'>最低1つ</font>示してください。

バギングは入力データの選び方を多様化する方法です。学習データから重複を許した上でランダムに抜き出すことで、N種類のサブセット（ブートストラップサンプル）を作り出します。それらによってモデルをN個学習し、推定結果の平均をとります。ブレンディングと異なり、それぞれの重み付けを変えることはありません。

[sklearn.model_selection.train_test_split — scikit-learn 0.20.0 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

scikit-learnのtrain_test_splitを、shuffleパラメータをTrueにして使うことで、ランダムにデータを分割することができます。これによりブートストラップサンプルが手に入ります。

推定結果の平均をとる部分はブースティングと同様の実装になります。

In [16]:
#予測した結果を足す0配列
bagging_linear = np.zeros(len(y_test_std)).reshape(-1, 1)

for i in range(20):
    bagging_X_train, _, bagging_y_train, _ = train_test_split(X_train_std, y_train_std, shuffle=True, test_size=0.2)
    
    #学習、予測
    y_pred, _, _ = pipe_score(bagging_X_train, X_test_std, bagging_y_train, y_test_std, LinearRegression())
    
    #予測した結果を配列に足し上げる
    bagging_linear += y_pred.reshape(-1, 1)
    
    #足し上げた結果をモデルの数で割る
    linear_bagging_y_pred = bagging_linear / (i+1)

    #MSE
    linear_bagging_mse = mean_squared_error(y_test_std, linear_bagging_y_pred)

    #R2
    linear_bagging_r2 = r2_score(y_test_std, linear_bagging_y_pred)
    
    print('{}番目 MSE : {:.3f} \t R2 : {:.3f}'.format(i+1, linear_bagging_mse, linear_bagging_r2))

1番目 MSE : 0.044 	 R2 : 0.711
2番目 MSE : 0.044 	 R2 : 0.712
3番目 MSE : 0.044 	 R2 : 0.712
4番目 MSE : 0.044 	 R2 : 0.712
5番目 MSE : 0.044 	 R2 : 0.712
6番目 MSE : 0.044 	 R2 : 0.712
7番目 MSE : 0.044 	 R2 : 0.712
8番目 MSE : 0.044 	 R2 : 0.712
9番目 MSE : 0.044 	 R2 : 0.712
10番目 MSE : 0.044 	 R2 : 0.712
11番目 MSE : 0.044 	 R2 : 0.712
12番目 MSE : 0.044 	 R2 : 0.713
13番目 MSE : 0.044 	 R2 : 0.713
14番目 MSE : 0.044 	 R2 : 0.713
15番目 MSE : 0.044 	 R2 : 0.713
16番目 MSE : 0.044 	 R2 : 0.713
17番目 MSE : 0.044 	 R2 : 0.713
18番目 MSE : 0.044 	 R2 : 0.713
19番目 MSE : 0.044 	 R2 : 0.713
20番目 MSE : 0.044 	 R2 : 0.713


In [17]:
#予測した結果を足す0配列
bagging_svm = np.zeros(len(y_test_std)).reshape(-1, 1)

for i in range(20):
    bagging_X_train, _, bagging_y_train, _ = train_test_split(X_train_std, y_train_std, shuffle=True, test_size=0.2)
    
    #学習、予測
    y_pred, _, _ = pipe_score(bagging_X_train, X_test_std, np.ravel(bagging_y_train), np.ravel(y_test_std), SVR(gamma='auto'))
    
    #予測した結果を配列に足し上げる
    bagging_svm += y_pred.reshape(-1, 1)
    
    #足し上げた結果をモデルの数で割る
    svm_bagging_y_pred = bagging_svm / (i+1)

    #MSE
    svm_bagging_mse = mean_squared_error(y_test_std, svm_bagging_y_pred)

    #R2
    svm_bagging_r2 = r2_score(y_test_std, svm_bagging_y_pred)
    
    print('{}番目 MSE : {:.3f} \t R2 : {:.3f}'.format(i+1, svm_bagging_mse, svm_bagging_r2))

1番目 MSE : 0.055 	 R2 : 0.639
2番目 MSE : 0.055 	 R2 : 0.641
3番目 MSE : 0.055 	 R2 : 0.642
4番目 MSE : 0.055 	 R2 : 0.641
5番目 MSE : 0.055 	 R2 : 0.641
6番目 MSE : 0.055 	 R2 : 0.641
7番目 MSE : 0.055 	 R2 : 0.641
8番目 MSE : 0.055 	 R2 : 0.641
9番目 MSE : 0.055 	 R2 : 0.640
10番目 MSE : 0.055 	 R2 : 0.640
11番目 MSE : 0.055 	 R2 : 0.641
12番目 MSE : 0.055 	 R2 : 0.641
13番目 MSE : 0.055 	 R2 : 0.641
14番目 MSE : 0.055 	 R2 : 0.641
15番目 MSE : 0.055 	 R2 : 0.641
16番目 MSE : 0.055 	 R2 : 0.640
17番目 MSE : 0.055 	 R2 : 0.640
18番目 MSE : 0.055 	 R2 : 0.640
19番目 MSE : 0.055 	 R2 : 0.640
20番目 MSE : 0.055 	 R2 : 0.640


In [22]:
#予測した結果を足す0配列
bagging_tree = np.zeros(len(y_test_std)).reshape(-1, 1)

for i in range(20):
    bagging_X_train, _, bagging_y_train, _ = train_test_split(X_train_std, y_train_std, shuffle=True, test_size=0.2)
    
    #学習、予測
    y_pred, _, _ = pipe_score(bagging_X_train, X_test_std, bagging_y_train, y_test_std, DecisionTreeRegressor())
    
    #予測した結果を配列に足し上げる
    bagging_tree += y_pred.reshape(-1, 1)
    
    #足し上げた結果をモデルの数で割る
    tree_bagging_y_pred = bagging_tree / (i+1)

    #MSE
    tree_bagging_mse = mean_squared_error(y_test_std, tree_bagging_y_pred)

    #R2
    tree_bagging_r2 = r2_score(y_test_std, tree_bagging_y_pred)
    
    print('{}番目 MSE : {:.3f} \t R2 : {:.3f}'.format(i+1, tree_bagging_mse, tree_bagging_r2))

1番目 MSE : 0.070 	 R2 : 0.546
2番目 MSE : 0.058 	 R2 : 0.622
3番目 MSE : 0.056 	 R2 : 0.637
4番目 MSE : 0.053 	 R2 : 0.654
5番目 MSE : 0.054 	 R2 : 0.651
6番目 MSE : 0.052 	 R2 : 0.661
7番目 MSE : 0.052 	 R2 : 0.661
8番目 MSE : 0.052 	 R2 : 0.660
9番目 MSE : 0.052 	 R2 : 0.661
10番目 MSE : 0.051 	 R2 : 0.669
11番目 MSE : 0.050 	 R2 : 0.671
12番目 MSE : 0.050 	 R2 : 0.672
13番目 MSE : 0.050 	 R2 : 0.675
14番目 MSE : 0.049 	 R2 : 0.679
15番目 MSE : 0.049 	 R2 : 0.678
16番目 MSE : 0.049 	 R2 : 0.678
17番目 MSE : 0.050 	 R2 : 0.677
18番目 MSE : 0.049 	 R2 : 0.679
19番目 MSE : 0.049 	 R2 : 0.681
20番目 MSE : 0.049 	 R2 : 0.683


各モデルにおいて、ブーストラップサンプルを1~20種類まで抜き出し、MSEとR2を求めた。線形回帰とSVMでは、ブーストラップサンプルの個数に関わらず精度は変わらなかったが、決定木のみ、ブーストラップサンプルを増やすにつれて精度が向上した。決定木単一の場合はMSEが0.067、R2が0.564であったが、20回抜き出した時はMSEは0.049、R2は0.683であった。
決定木の精度が向上した理由として、決定木は訓練データが異なると、ツリー構造が異なってくる可能性があるが、バギングを行うことで訓練データの偏りがなくなったことで、精度が向上したと思われる。

## 【問題3】スタッキング

スタッキングを実装し、単一モデルより精度があがる例を最低1つ示してください。

スタッキングの手順は以下の通りです。最低限ステージ0とステージ1があればスタッキングは成立するため、それを実装してください。まずは$K_0=3,M_0=2$程度にします。

学習時

（ステージ$0$）

- 学習データを$K_0$個に分割する。

- 分割した内の$(K_0 - 1)$個をまとめて学習用データ、残り$1$個を推定用データとする組み合わせが$K_0$個作れる。

- あるモデルのインスタンスを$K_0$個用意し、異なる学習用データを使い学習する。

- それぞれの学習済みモデルに対して、使っていない残り$1$個の推定用データを入力し、推定値を得る。（これをブレンドデータと呼ぶ）

- さらに、異なるモデルのインスタンスも$K_0$個用意し、同様のことを行う。モデルが$M_0$個あれば、 $M_0$個のブレンドデータが得られる。

(ステージ$n$）

- ステージ$n - 1$のブレンドデータを$M_{n - 1}$次元の特徴量を持つ学習用データと考え、 $K_n$個に分割する。以下同様である。

（ステージ$N$）＊最後のステージ

- ステージ$N - 1$の$M_{N - 1}$個のブレンドデータを$M_{N - 1}$次元の特徴量の入力として、1種類のモデルの学習を行う。これが最終的な推定を行うモデルとなる。

推定時

（ステージ$0$）

- テストデータを$K_0 \times M_0$個の学習済みモデルに入力し、$K_0 \times M_0$個の推定値を得る。これを$K_0$の軸で平均値を求め$M_0$次元の特徴量を持つデータを得る。（ブレンドテストと呼ぶ）

（ステージ$n$）

- ステージ$n - 1$で得たブレンドテストを$K_n \times M_n$個の学習済みモデルに入力し、$K_n \times M_n$個の推定値を得る。これを$K_n$の軸で平均値を求め$M_0$次元の特徴量を持つデータを得る。（ブレンドテストと呼ぶ）

（ステージ$N$）＊最後のステージ

- ステージ$N - 1$で得たブレンドテストを学習済みモデルに入力し、推定値を得る。

 **ステージ0**

In [23]:
#線形回帰

#空の配列を用意
stac_linear_array = np.array([])
#テストデータを格納する0配列
stac_linear_test_array = np.zeros([len(y_test_std), 3])

#学習データを3分割
kf = KFold(n_splits=3)

#分割したデータをリストへ格納
for i, (train, test) in enumerate(kf.split(X_train_std, y_train_std)):
    
    #インスタンス化
    stac_linear = LinearRegression()
    
    #分割した学習用データで学習
    stac_linear.fit(X_train_std[train], y_train_std[train])
    
    #使用していない推定データで推定
    y_pred_linear = stac_linear.predict(X_train_std[test])
    
    #テストデータを推定
    y_pred_test_linear = stac_linear.predict(X_test_std)
    
    #予測した結果をnumpy配列に入れる
    stac_linear_array = np.append(stac_linear_array, y_pred_linear).reshape(-1, 1)
    stac_linear_test_array[:, i] += y_pred_test_linear

In [24]:
#SVM

#空の配列を用意
stac_svm_array = np.array([])
#テストデータを格納する0配列
stac_svm_test_array = np.zeros([len(y_test_std), 3])

#分割したデータをリストへ格納
for i, (train, test) in enumerate(kf.split(X_train_std, y_train_std)):
    
    #インスタンス化
    stac_svm = SVR(gamma='auto')
    
    #分割した学習用データで学習
    stac_svm.fit(X_train_std[train], y_train_std[train])
    
    #使用していない推定データで推定
    y_pred_svm = stac_svm.predict(X_train_std[test])
    
    #テストデータを推定
    y_pred_test_svm = stac_linear.predict(X_test_std)
    
    #予測した結果をnumpy配列に入れる
    stac_svm_array = np.append(stac_svm_array, y_pred_svm).reshape(-1, 1)
    stac_svm_test_array[:, i] += y_pred_test_svm

In [25]:
#決定木

#空の配列を用意
stac_tree_array = np.array([])
#テストデータを格納する0配列
stac_tree_test_array = np.zeros([len(y_test_std), 3])

#分割したデータをリストへ格納
for i, (train, test) in enumerate(kf.split(X_train_std, y_train_std)):
    
    #インスタンス化
    stac_tree = DecisionTreeRegressor()
    
    #分割した学習用データで学習
    stac_tree.fit(X_train_std[train], y_train_std[train])
    
    #使用していない推定データで推定
    y_pred_tree = stac_tree.predict(X_train_std[test])
    
    #テストデータを推定
    y_pred_test_tree = stac_tree.predict(X_test_std)
    
    #予測した結果をnumpy配列に入れる
    stac_tree_array = np.append(stac_tree_array, y_pred_tree).reshape(-1, 1)
    stac_tree_test_array[:, i] += y_pred_test_tree

In [26]:
#各モデルのブレンドデータを横に結合
data0 = np.hstack([stac_linear_array, stac_svm_array, stac_tree_array])

In [27]:
#モデルごとに平均値をとり、各モデルのブレンドテストを横に結合
data0_test = np.hstack([stac_linear_test_array.mean(axis=1).reshape(-1, 1), 
                                            stac_svm_test_array.mean(axis=1).reshape(-1, 1), 
                                            stac_tree_test_array.mean(axis=1).reshape(-1, 1)])

**ステージ1**

In [28]:
#1種類のモデルで学習
#インスタンス化
stac1 = LinearRegression()

#学習
stac1.fit(data0, y_train_std)

#テストデータで予測
stac_pred = stac1.predict(data0_test)

#MSE
stac_mse = mean_squared_error(y_test_std, stac_pred)

#R2
stac_r2 = r2_score(y_test_std, stac_pred)

print('MSE :', stac_mse, '\t', 'R2 :', stac_r2)

MSE : 0.041653843466348754 	 R2 : 0.7283283237370972


スタッキングはステージ0とステージ1を実装した。結果はMSEが0.042、R2が0.728であり、単一モデルで一番精度の良い線形回帰のMSE:0.044、R2:0.713より精度が少し高かった。